# Pipeline de Detecção de Objetos usando GStreamer e DeepStream

Este notebook ensina como criar um pipeline de detecção de objetos utilizando GStreamer e o framework DeepStream da NVIDIA. O pipeline irá processar a entrada de vídeo de uma câmera usb, realizar a detecção de objetos com Deep Learning e exibir os resultados com sobreposições indicando o número e o tipo de objetos detectados.

Ferramentas:

- GStreamer: https://github.com/GStreamer/gstreamer
- DeepStream: https://developer.nvidia.com/deepstream-sdk

**OBS**: Para que seja feito em Python, é necessário compilar as bindings do DeepStream para Python. Esse link contém as intruções de como o fazer: https://github.com/NVIDIA-AI-IOT/deepstream_python_apps/tree/master/bindings

## Importação das bibliotecas necessárias

In [ ]:
# Importação de bibliotecas necessárias
import os
import sys
import logging
from datetime import datetime

# Adicionando o caminho do módulo necessário ao sistema
sys.path.append('../')

# Importando as bibliotecas GStreamer
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst

# Configurando o logging para informações detalhadas
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Importando outras bibliotecas necessárias
import pandas as pd
import pyds
from common.is_aarch_64 import is_aarch64
from common.bus_call import bus_call

## Variáveis Globais de Configuração

In [ ]:
PGIE_FILE = os.environ.get('PGIE_CONFIG', './config_infer_primary_yoloV8.txt')
WIDTH = os.environ.get('WIDTH', 1920)
HEIGHT = os.environ.get('HEIGHT', 1080)
FPS = os.environ.get('FPS', 30)

## Funções Auxiliares

### Carregar o Arquivo de Configuração

Esta função carrega o arquivo de configuração PGIE e extrai o caminho do arquivo de rótulos. O arquivo de configuração de inferência PGIE é necessário para informar ao pipeline quais são as configurações do modelo de deep learning que será utilizado para a detecção de objetos.



In [ ]:
def load_config_file(config_file_path):
    """
    Carrega o arquivo de configuração PGIE e extrai o caminho do arquivo de rótulos.

    Args:
        config_file_path (str): Caminho para o arquivo de configuração PGIE.

    Returns:
        str: Caminho para o arquivo de rótulos.

    Raises:
        ValueError: Se o caminho do arquivo de rótulos não for encontrado no arquivo de configuração.
    """
    with open(config_file_path, 'r') as config_file:
        lines = config_file.readlines()
    for line in lines:
        if 'labelfile-path=' in line:
            return line.split('=')[1].strip()
    raise ValueError("Caminho do arquivo de rótulos não encontrado no arquivo de configuração")

### Carregar o Arquivo de Rótulos

In [ ]:
def load_label_file(label_file_path):
    """
    Carrega os rótulos do arquivo de rótulos e cria um dicionário que mapeia nomes de classes para seus respectivos IDs.

    Args:
        label_file_path (str): Caminho para o arquivo de rótulos.

    Returns:
        dict: Dicionário que mapeia nomes de classes para seus respectivos IDs.
    """
    with open(label_file_path, 'r') as label_file:
        lines = label_file.read().splitlines()
    return {line: i for i, line in enumerate(lines)}

## Criação do Pipeline de Visão

A função `create_pipeline` é responsável por criar e configurar os elementos do pipeline GStreamer. Essa função inicializa todos os elementos necessários para o pipeline de processamento de vídeo e garante que todos os elementos sejam criados corretamente.

#### Etapas da Função:
1. **Criação do Pipeline**:
   - Cria o objeto pipeline usando `Gst.Pipeline()`.
   - Verifica se o pipeline foi criado com sucesso. Caso contrário, lança uma exceção `RuntimeError`.

2. **Criação dos Elementos do Pipeline**:
   - **Fonte de Vídeo (`v4l2src`)**: Elemento que captura vídeo de um dispositivo V4L2.
   - **Filtro de Capacidades (`caps_v4l2src`)**: Filtro que define as capacidades do vídeo capturado.
   - **Conversor de Vídeo (`vidconvsrc`)**: Elemento que converte o formato do vídeo.
   - **Conversor de Vídeo NVidia (`nvvidconvsrc`)**: Elemento que converte o formato do vídeo para um formato compatível com NVMM.
   - **Filtro de Capacidades NVMM (`caps_vidconvsrc`)**: Filtro que define as capacidades do vídeo na memória NVMM.
   - **Muxer de Fluxo (`streammux`)**: Elemento que agrupa vários fluxos de entrada em um único fluxo de saída.
   - **Inferenciador Primário (`pgie`)**: Elemento que realiza a inferência do modelo de deep learning.
   - **Conversor de Vídeo NVidia (`nvvidconv`)**: Elemento que converte o formato do vídeo após a inferência.
   - **Display On-Screen (`nvosd`)**: Elemento que exibe metadados, como caixas delimitadoras e texto, sobre o vídeo.
   - **Sink de Vídeo (`sink`)**: Elemento que renderiza o vídeo. Dependendo da arquitetura, usa `nv3dsink` ou `nveglglessink`.

3. **Verificação da Criação dos Elementos**:
   - Itera sobre todos os elementos criados e verifica se cada elemento foi criado com sucesso.
   - Se algum elemento não puder ser criado, lança uma exceção `RuntimeError` com o nome do elemento que falhou.

4. **Retorno dos Elementos**:
   - Retorna um dicionário contendo todos os elementos criados.

#### Elementos Criados:
- **pipeline**: Pipeline GStreamer.
- **source**: Fonte de vídeo (`v4l2src`).
- **caps_v4l2src**: Filtro de capacidades para a fonte de vídeo.
- **vidconvsrc**: Conversor de vídeo.
- **nvvidconvsrc**: Conversor de vídeo NVidia.
- **caps_vidconvsrc**: Filtro de capacidades para NVMM.
- **streammux**: Muxer de fluxo.
- **pgie**: Inferenciador primário.
- **nvvidconv**: Conversor de vídeo NVidia.
- **nvosd**: Display on-screen.
- **sink**: Sink de vídeo (`nv3dsink` ou `nveglglessink`).



In [ ]:
# Função para criar e configurar os elementos do pipeline

def create_pipeline():
    """
    Cria e configura o pipeline GStreamer e seus elementos.

    Returns:
        dict: Dicionário contendo os elementos criados do GStreamer.

    Raises:
        RuntimeError: Se algum elemento do GStreamer não puder ser criado.
    """
    pipeline = Gst.Pipeline()
    if not pipeline:
        raise RuntimeError("Não foi possível criar o Pipeline")

    source = Gst.ElementFactory.make("v4l2src", "usb-cam-source")
    caps_v4l2src = Gst.ElementFactory.make("capsfilter", "v4l2src_caps")
    vidconvsrc = Gst.ElementFactory.make("videoconvert", "convertor_src1")
    nvvidconvsrc = Gst.ElementFactory.make("nvvideoconvert", "convertor_src2")
    caps_vidconvsrc = Gst.ElementFactory.make("capsfilter", "nvmm_caps")
    streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
    pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
    nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
    nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
    sink = Gst.ElementFactory.make("nv3dsink" if is_aarch64() else "nveglglessink", "nvvideo-renderer")

    elements = {
        "pipeline": pipeline,
        "source": source,
        "caps_v4l2src": caps_v4l2src,
        "vidconvsrc": vidconvsrc,
        "nvvidconvsrc": nvvidconvsrc,
        "caps_vidconvsrc": caps_vidconvsrc,
        "streammux": streammux,
        "pgie": pgie,
        "nvvidconv": nvvidconv,
        "nvosd": nvosd,
        "sink": sink
    }
    
    for name, element in elements.items():
        if not element:
            raise RuntimeError(f"Não foi possível criar o {name}")
    
    return elements

### Ligar os Elementos do Pipeline

Esta função liga os elementos do GStreamer dentro do pipeline, estabelecendo as conexões necessárias entre eles para o fluxo de dados funcionar corretamente.

In [ ]:
# Função para ligar os elementos do pipeline

def link_elements(elements):
    """
    Liga os elementos do GStreamer dentro do pipeline.

    Args:
        elements (dict): Dicionário contendo os elementos do GStreamer.

    Returns:
        None

    Raises:
        RuntimeError: Se algum elemento não puder ser ligado.
    """
    elements["pipeline"].add(elements["source"])
    elements["pipeline"].add(elements["caps_v4l2src"])
    elements["pipeline"].add(elements["vidconvsrc"])
    elements["pipeline"].add(elements["nvvidconvsrc"])
    elements["pipeline"].add(elements["caps_vidconvsrc"])
    elements["pipeline"].add(elements["streammux"])
    elements["pipeline"].add(elements["pgie"])
    elements["pipeline"].add(elements["nvvidconv"])
    elements["pipeline"].add(elements["nvosd"])
    elements["pipeline"].add(elements["sink"])
    
    elements["source"].link(elements["caps_v4l2src"])
    elements["caps_v4l2src"].link(elements["vidconvsrc"])
    elements["vidconvsrc"].link(elements["nvvidconvsrc"])
    elements["nvvidconvsrc"].link(elements["caps_vidconvsrc"])
    
    sinkpad = elements["streammux"].get_request_pad("sink_0")
    srcpad = elements["caps_vidconvsrc"].get_static_pad("src")
    
    srcpad.link(sinkpad)
    elements["streammux"].link(elements["pgie"])
    elements["pgie"].link(elements["nvvidconv"])
    elements["nvvidconv"].link(elements["nvosd"])
    elements["nvosd"].link(elements["sink"])

### Configurar Propriedades dos Elementos

A função `set_element_properties` é responsável por configurar as propriedades dos elementos do GStreamer. Essa configuração é essencial para garantir que cada elemento no pipeline funcione corretamente de acordo com os requisitos do aplicativo.

#### Etapas da Função:
1. **Configuração do Elemento `caps_v4l2src`**:
   - Define a propriedade `caps` para filtrar o formato de vídeo e a taxa de frames.
   - Utiliza `Gst.Caps.from_string(f"video/x-raw, framerate={FPS}/1")` para definir as capacidades do vídeo cru com a taxa de frames especificada.

2. **Configuração do Elemento `caps_vidconvsrc`**:
   - Define a propriedade `caps` para filtrar o formato de vídeo na memória NVMM.
   - Utiliza `Gst.Caps.from_string("video/x-raw(memory:NVMM)")` para definir as capacidades de vídeo na memória NVMM.

3. **Configuração do Elemento `source`**:
   - Define a propriedade `device` para especificar o caminho do dispositivo V4L2 ( camera USB ).
   - Utiliza `elements["source"].set_property('device', device_path)` para definir o caminho do dispositivo.

4. **Configuração do Elemento `streammux`**:
   - Define várias propriedades do muxer de fluxo:
     - `width`: Define a largura do fluxo de vídeo.
     - `height`: Define a altura do fluxo de vídeo.
     - `batch-size`: Define o tamanho do batch para processamento em batch.
     - `batched-push-timeout`: Define o tempo limite para envio em batch.
   - Utiliza `elements["streammux"].set_property` para definir essas propriedades.

5. **Configuração do Elemento `pgie`**:
   - Define a propriedade `config-file-path` para especificar o caminho do arquivo de configuração do inferenciador primário.
   - Utiliza `elements["pgie"].set_property('config-file-path', PGIE_FILE)` para definir o caminho do arquivo de configuração.

6. **Configuração do Elemento `sink`**:
   - Define a propriedade `sync` para desativar a sincronização do vídeo.
   - Utiliza `elements["sink"].set_property('sync', False)` para definir essa propriedade.

#### Propriedades Utilizadas:
- **caps_v4l2src**:
  - `caps`: Capacidades do vídeo cru com a taxa de frames especificada.
  
- **caps_vidconvsrc**:
  - `caps`: Capacidades do vídeo na memória NVMM.
  
- **source**:
  - `device`: Caminho do dispositivo V4L2.
  
- **streammux**:
  - `width`: Largura do fluxo de vídeo.
  - `height`: Altura do fluxo de vídeo.
  - `batch-size`: Tamanho do batch para processamento.
  - `batched-push-timeout`: Tempo limite para envio em batch.
  
- **pgie**:
  - `config-file-path`: Caminho do arquivo PGIE.
  
- **sink**:
  - `sync`: Desativa a sincronização do vídeo.



In [ ]:
def set_element_properties(elements, device_path):
    """
    Configura as propriedades dos elementos do GStreamer.

    Args:
        elements (dict): Dicionário contendo os elementos do GStreamer.
        device_path (str): Caminho para o dispositivo V4L2.

    Returns:
        None
    """
    elements["caps_v4l2src"].set_property('caps', Gst.Caps.from_string(f"video/x-raw, framerate={FPS}/1"))
    elements["caps_vidconvsrc"].set_property('caps', Gst.Caps.from_string("video/x-raw(memory:NVMM)"))
    elements["source"].set_property('device', device_path)
    elements["streammux"].set_property('width', WIDTH)
    elements["streammux"].set_property('height', HEIGHT)
    elements["streammux"].set_property('batch-size', 1)
    elements["streammux"].set_property('batched-push-timeout', 4000000)
    elements["pgie"].set_property('config-file-path', PGIE_FILE)
    elements["sink"].set_property('sync', False)

## Função de Probe

A função `osd_sink_pad_buffer_probe` é uma função de probe usada para lidar com a extração de metadados e atualizações de exibição. Esta função é chamada sempre que um buffer passa pelo pad ao qual a probe está anexada. 

#### Explicação da Função:
1. **Carregamento do arquivo de rótulos**:
   - A função começa carregando o caminho dos arquivos de configuração do modelo de DeepLeaning e parâmetros de video `load_config_file(PGIE_FILE)` e de anotações dos dados  `load_label_file(label_file_path)`.

2. **Extração do buffer**:
   - Obtém o buffer GStreamer (explicado abaixo) usando `info.get_buffer()`. Se o buffer não estiver presente, a função retorna `Gst.PadProbeReturn.OK`.

3. **Extração de Metadados**:
   - Usa `pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))` para obter os metadados do batch de imagens.
   - Obtém a lista de metadados do frame usando `batch_meta.frame_meta_list`.

4. **Processamento de Cada frame**:
   - Itera sobre cada frame no batch.
   - Usa `pyds.NvDsFrameMeta.cast(l_frame.data)` para converter os dados do frame em metadados do frame.
   - Chama a função `process_frame` para processar os metadados do frame.
   - Move para o próximo frame na lista.

#### Metadados Utilizados:
- **NvDsBatchMeta**: Metadados do batch que contêm informações sobre todos os frames processados em um único batch.
- **NvDsFrameMeta**: Metadados do frame que contêm informações específicas sobre cada frame, incluindo número do frame, número de objetos detectados, e a lista de objetos.
- **NvDsObjectMeta**: Metadados do objeto que contêm informações sobre cada objeto detectado, incluindo ID da classe do objeto e coordenadas da caixa delimitadora.

#### Documentação Adicional:
Para mais informações sobre os metadados utilizados no DeepStream, consulte a [documentação oficial da NVIDIA](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_metadata.html).



In [ ]:
def osd_sink_pad_buffer_probe(pad, info, u_data):
    """
    Função de probe para lidar com a extração de metadados e atualizações de exibição.

    Args:
        pad: O pad ao qual a probe está anexada.
        info: Informações do buffer.
        u_data: Dados do usuário (não utilizados).

    Returns:
        Gst.PadProbeReturn: Status da operação da probe.
    """
    label_file_path = load_config_file(PGIE_FILE)
    object_classes = load_label_file(label_file_path)
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        return Gst.PadProbeReturn.OK
    
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    
    # Processando cada frame no batch
    while l_frame is not frame:
        try:
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        process_frame(frame_meta, object_classes, batch_meta)
        
        try:
            l_frame = l_frame.next
        except StopIteration:
            break
    
    return Gst.PadProbeReturn.OK

### Função para Processar cada Frame

A função `process_frame` é responsável por processar cada frame, contando os objetos detectados e atualizando os metadados de exibição. Esta função é essencial para a contagem de objetos em cada frame e a exibição dessas informações na tela.

#### Argumentos:
- `frame_meta`: Metadados do frame.
- `object_classes (dict)`: Dicionário que mapeia nomes de classes para IDs.
- `batch_meta`: Metadados do batch.

#### Retorno:
- `None`

#### Explicação da Função:
1. **Inicialização do Contador de Objetos**:
   - Cria um dicionário `obj_counter` para contar o número de objetos detectados para cada classe.

2. **Extração de Informações do Frame**:
   - Obtém o número do frame usando `frame_meta.frame_num`.
   - Obtém o número de objetos detectados no frame usando `frame_meta.num_obj_meta`.
   - Obtém a lista de metadados de objetos usando `frame_meta.obj_meta_list`.

3. **Contagem dos Objetos Detectados**:
   - Itera sobre a lista de objetos e conta quantos objetos de cada classe foram detectados.
   - Usa `pyds.NvDsObjectMeta.cast(l_obj.data)` para converter os dados do objeto em metadados do objeto.
   - Atualiza o contador de objetos `obj_counter`.

4. **Criação e Configuração dos Metadados de Exibição**:
   - Obtém a hora atual usando `datetime.now().strftime("%H:%M:%S")`.
   - Adquire um objeto de metadados de exibição usando `pyds.nvds_acquire_display_meta_from_pool(batch_meta)`.
   - Define o número de rótulos de exibição (`display_meta.num_labels = 1`).
   - Cria a string de texto de exibição que inclui o número do frame e o número de objetos detectados.

5. **Adição de Informações de Contagem de Objetos à Exibição**:
   - Itera sobre o dicionário `obj_counter` e adiciona a contagem de objetos de cada classe à string de texto de exibição.
   - Configura os parâmetros de texto, como posição, fonte e cor do texto, e cor de fundo.

6. **Adição dos Metadados de Exibição ao Frame**:
   - Adiciona os metadados de exibição ao frame usando `pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)`.

In [ ]:
def process_frame(frame_meta, object_classes, batch_meta):
    """
    Processa cada frame, conta os objetos e atualiza os metadados de exibição.

    Args:
        frame_meta: Metadados do frame.
        object_classes (dict): Dicionário que mapeia nomes de classes para IDs.
        batch_meta: Metadados do batch.

    Returns:
        None
    """
    obj_counter = {id: 0 for _, id in object_classes.items()}
    frame_number = frame_meta.frame_num
    num_rects = frame_meta.num_obj_meta
    l_obj = frame_meta.obj_meta_list

    # Contando os objetos detectados no frame
    while l_obj is not None:
        try:
            obj_meta = pyds.NvDsObjectMeta.cast(l_obj.data)
        except StopIteration:
            break
        obj_counter[obj_meta.class_id] += 1
        try:
            l_obj = l_obj.next
        except StopIteration:
            break

    current_time = datetime.now().strftime("%H:%M:%S")
    display_meta = pyds.nvds_acquire_display_meta_from_pool(batch_meta)
    display_meta.num_labels = 1
    py_nvosd_text_params = display_meta.text_params[0]
    display_txt = f"Frame Number={frame_number} Number of Objects={num_rects}"
    
    # Adicionando informações de contagem de objetos à exibição
    for k, v in obj_counter.items():
        _class = [cls for cls in object_classes.keys() if object_classes[cls] == k][0]
        display_txt += f" {_class}_count={v}"
    
    py_nvosd_text_params.display_text = display_txt
    py_nvosd_text_params.x_offset = 10
    py_nvosd_text_params.y_offset = 12
    py_nvosd_text_params.font_params.font_name = "Serif"
    py_nvosd_text_params.font_params.font_size = 10
    py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
    py_nvosd_text_params.set_bg_clr = 1
    py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
    pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)

## Função Principal

### Função Principal para Inicializar o Pipeline e o Loop de Eventos

Esta é a função principal que analisa os argumentos, configura o pipeline e inicia o loop de eventos. Ela inicializa o GStreamer, cria o loop de eventos GLib, configura o pipeline e o inicia, além de configurar o bus para receber mensagens de erro e estado do pipeline.

### GStreamer:
- **Biblioteca multimídia**: GStreamer é uma biblioteca que permite a construção de gráficos de fluxo de mídia complexos.
- **Elementos e plugins**: Utiliza uma variedade de elementos (plugins) que podem ser conectados para formar pipelines de processamento de mídia.
- **Flexibilidade**: Suporta diferentes tipos de mídia e pode ser utilizado para processamento de áudio e vídeo em tempo real ou offline.

### GLib:
- **Fundação para GTK+**: GLib é a biblioteca base usada pelo GTK+, mas também pode ser utilizada de forma independente.
- **Utilidades de sistema**: Fornece funções para manipulação de estruturas de dados, utilidades portáveis de entrada/saída e interfaces para threads.
- **Loop de eventos**: GLib Main Loop é uma implementação de loop de eventos que permite a integração de diferentes fontes de eventos (como timers e I/O) em um único loop.

### GTK:
- **Biblioteca de interface gráfica**: GTK (GIMP Toolkit) é uma biblioteca para criar interfaces gráficas de usuário (GUI) de maneira multiplataforma.
- **Aplicações**: Utilizada para desenvolver uma variedade de aplicações gráficas, desde simples utilitários até softwares complexos.
- **Integração com GLib**: Baseada na GLib, fornece uma coleção de widgets (como botões, janelas, sliders) para facilitar o desenvolvimento de interfaces gráficas.


In [ ]:
def main(args):
    """
    Função principal para analisar argumentos, configurar o pipeline e iniciar o loop de eventos.

    Args:
        args (list): Argumentos da linha de comando.

    Returns:
        int: Status de saída.
    """

    Gst.init(None)
    loop = GLib.MainLoop()
    
    # Caminho do dispositivo de vídeo (padrão: /dev/video0)
    device_path = "/dev/video0" if len(args) < 2 else args[1]
    elements = create_pipeline()
    link_elements(elements)
    set_element_properties(elements, device_path)
    
    # Configurando o bus para receber mensagens
    bus = elements["pipeline"].get_bus()
    bus.add_signal_watch()
    bus.connect("message", bus_call, loop)
    
    # Adicionando a função de probe ao pad do OSD
    osdsinkpad = elements["nvosd"].get_static_pad("sink")
    osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)
    
    logger.info("Iniciando o pipeline")
    elements["pipeline"].set_state(Gst.State.PLAYING)
    
    try:
        loop.run()
    except:
        pass
    finally:
        elements["pipeline"].set_state(Gst.State.NULL)

if __name__ == '__main__':
    sys.exit(main(sys.argv))